In [1]:
import matplotlib.pyplot as plt
# import matplotlib as mpl
%matplotlib inline

import cPickle as pickle
import itertools
import pandas as pd
# import networkx as nx

import numpy as np
# from numpy.random import rand

# from scipy.io import loadmat
from scipy.ndimage import morphology
# from scipy.cluster.vq import kmeans,vq
from scipy import signal, ndimage


# from make_map import make_map
# from file_util import load_matlab_data
# from plot import plot_discharge
from metrics import *
# from stratigraphy import sedimentograph
from spatial_dist import *


from shapely.geometry import shape, Point, Polygon, MultiLineString, MultiPoint, MultiPolygon, LineString
from shapely.ops import polygonize_full #, transform

import rasterio
# from rasterio.features import shapes
from osgeo import ogr, gdal, osr
import fiona

import clusterpy
import skimage
# from sklearn.preprocessing import MinMaxScaler

from metrics_scripts.metrics_utils import *


def SaveRaster(array, filename, r_xmin, r_ymin, res, downsampling = 1):
    '''
    Saves a numpy array into a raster file
    '''

    rasterOrigin = (r_xmin, r_ymin)
    pixelWidth = res * downsampling
    pixelHeight = res * downsampling

    reversed_arr = array[::-1] # reverse array so the tif looks like the array
    array2raster(filename,rasterOrigin,pixelWidth,pixelHeight,reversed_arr) # convert array to raster
    
    

def array2raster(newRasterfn, rasterOrigin, pixelWidth, pixelHeight, array, spatial_ref = 32645):
    '''
    Converts numpy array into raster, given a model raster file
    '''

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]
    
    output_raster = gdal.GetDriverByName('GTiff').Create(newRasterfn, cols, rows, 1, gdal.GDT_Float32)
    output_raster.GetRasterBand(1).WriteArray( array ) 
    output_raster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(spatial_ref)
    output_raster.SetProjection(srs.ExportToWkt())
    output_raster.FlushCache()

    
    
def latlong_to_index(pt, xmin, xmax, ymin, ymax, array_size):
    '''
    Calculates index of point on array from latitude and longitude
    '''
    
    newx = int(array_size[1] * (pt[0] - xmin) / (xmax - xmin))
    newy = int(array_size[0] - array_size[0] * (pt[1] - ymin) / (ymax - ymin))
    
    return (newx, newy)



def create_shapefile_from_shapely_multi(features, filename,
                                        fields = {}, field_type = {},
                                        buffer_width = 0, spatial_ref = 32645):
    '''
    Creates a shapefile from a
    Shapely MultiPolygon, MultiLineString, or MultiPoint
    '''


    driver = ogr.GetDriverByName('Esri Shapefile')
    ds = driver.CreateDataSource(filename)

    srs = osr.SpatialReference()
    srs.ImportFromEPSG(spatial_ref)

    layer = ds.CreateLayer('', srs, ogr.wkbPolygon)

    for f in fields.keys():
        fieldDefn = ogr.FieldDefn(f, field_type[f])
        layer.CreateField(fieldDefn)

    defn = layer.GetLayerDefn()


    for i in range(len(features)):

        poly = features[i].buffer(buffer_width)

        # Create a new feature (attribute and geometry)
        feat = ogr.Feature(defn)

        for f in fields.keys():
            feat.SetField(f, fields[f][i])

        # Make a geometry from Shapely object
        geom = ogr.CreateGeometryFromWkb(poly.wkb)
        feat.SetGeometry(geom)

        layer.CreateFeature(feat)
        feat = geom = None  # destroy these


    # Save and close everything
    ds = layer = feat = geom = None

    
    
    
def create_tiff_from_shapefile(InputVector, OutputImage, RefImage):
    '''
    Rasterizes a shapefile
    '''

    gdalformat = 'GTiff'
    datatype = gdal.GDT_Byte
    burnVal = 1 #value for the output image pixels

    ##########################################################
    # Get projection info from reference image
    Image = gdal.Open(RefImage, gdal.GA_ReadOnly)

    # Open Shapefile
    Shapefile = ogr.Open(InputVector)
    Shapefile_layer = Shapefile.GetLayer()

    # Rasterise
    Output = gdal.GetDriverByName(gdalformat).Create(OutputImage,
                                                     Image.RasterXSize,
                                                     Image.RasterYSize,
                                                     1,
                                                     datatype,
                                                     options=['COMPRESS=DEFLATE'])
    Output.SetProjection(Image.GetProjectionRef())
    Output.SetGeoTransform(Image.GetGeoTransform()) 

    # Write data to band 1
    Band = Output.GetRasterBand(1)
    Band.SetNoDataValue(0)
    gdal.RasterizeLayer(Output, [1], Shapefile_layer, burn_values=[burnVal])

    # Close datasets
    Band = None
    Output = None
    Image = None
    Shapefile = None


In [2]:
def get_value_of_raster_within_polygons(raster, polygons, r_xmin, r_xmax, r_ymin, r_ymax, min_valid_label = None):
    '''
    Samples raster values at representative point of polygons
    '''
    
    poly_points_utm = [i.representative_point().coords[0] for i in polygons]
    poly_points_index = [latlong_to_index(i, r_xmin, r_xmax, r_ymin, r_ymax, raster.shape) for i in poly_points_utm]
    
    poly_points_value = [raster[i[::-1]] for i in poly_points_index]
    
    step = 10

    while (min_valid_label is not None) & (step < 25):

        bad_pts = np.where(np.array(poly_points_value) < min_valid_label)[0]

        if len(bad_pts) > 0:
            
            for i in bad_pts:

                pts = RegularGridSampling(polygons[i], step = step)
                pts_utm = [p.coords[0] for p in pts]
                pts_index = [latlong_to_index(p, r_xmin, r_xmax, r_ymin, r_ymax, raster.shape) for p in pts_utm]
                pts_value = [raster[p[::-1]] for p in pts_index]
                good_pts = np.where(np.array(pts_value) >= min_valid_label)[0]

                if len(good_pts) > 0:
                    poly_points_value[i] = pts_value[good_pts[0]]
                    poly_points_index[i] = pts_index[good_pts[0]]
                    
            step += 5
        else:
            step = 100
            
    bad_pts = np.where(np.array(poly_points_value) < min_valid_label)[0]
    
    return poly_points_value, poly_points_index, bad_pts
    

def RegularGridSampling(polygon, x_interval = None, y_interval = None, step = None):
    """
    Perform sampling by substituting the polygon with a regular grid of
    sample points within it. The distance between the sample points is
    given by x_interval and y_interval.
    """
    samples = []
    
    if step is None:
        step = 10
    
    if x_interval is None:
        x_interval = (polygon.bounds[2] - polygon.bounds[0]) / step
        
    if y_interval is None:   
        y_interval = (polygon.bounds[3] - polygon.bounds[1]) / step
    
    ll = polygon.bounds[:2]
    ur = polygon.bounds[2:]
    low_x = int(ll[0]) / x_interval * x_interval
    upp_x = int(ur[0]) / x_interval * x_interval + x_interval
    low_y = int(ll[1]) / y_interval * y_interval
    upp_y = int(ur[1]) / y_interval * y_interval + y_interval

    for x in np.arange(low_x, upp_x, x_interval):
        for y in np.arange(low_y, upp_y, y_interval):
            p = Point(x, y)
            if p.within(polygon):
                samples.append(p)
                
    
                
    return MultiPoint(samples)

def azimuth(point1, point2):
    '''azimuth between 2 shapely points (interval 0 - 360), from vertical'''
    
    angle = np.arctan2(point2.x - point1.x, point2.y - point1.y)
    az = np.degrees(angle) if angle > 0 else np.degrees(angle) + 360
    
    return az

In [3]:
# statistics for distributions

def mean_(val, freq):
    return np.average(val, weights = freq)

def median_(val, freq):
    order = np.argsort(val)
    cdf = np.cumsum(freq[order])
    return val[order][np.searchsorted(cdf, cdf[-1] // 2)]

def mode_(val, freq): #in the strictest sense, assuming unique mode
    return val[np.argmax(freq)]

def var_(val, freq):
    avg = mean_(val, freq)
    dev = freq * (val - avg) ** 2
    return dev.sum() / (freq.sum() - 1)

def std_(val, freq):
    return np.sqrt(var_(val, freq))

In [4]:
def Polygon_axes(polygon):
    '''
    Calculates major and minor axes, and major axis azimuth, for a Shapely polygon
    '''

    rect = polygon.minimum_rotated_rectangle.exterior

    len1 = LineString(rect.coords[0:2]).length
    len2 = LineString(rect.coords[1:3]).length

    if len1 > len2:
        orientation = azimuth(Point(rect.coords[0]), Point(rect.coords[1]))
    else:
        orientation = azimuth(Point(rect.coords[1]), Point(rect.coords[2]))
        
    if orientation > 180:
        orientation = orientation - 180

    axminor, axmajor = np.sort([len1, len2])

    return axminor, axmajor, orientation

In [5]:
def get_CDF(data):
    '''
    Calculate the Cumulative Distribution Function (CDF) of a 1D array of data
    '''

    data_sorted = np.flipud(np.sort(data))
    p = 1. * np.arange(len(data)) / (len(data) - 1)

    return [data_sorted, p]



# data_sorted, p = get_CDF(island_prop_array[:,24])

# plt.figure()
# plt.loglog(data_sorted, p)

In [6]:
def outline_to_mask(line, x, y):
    """
    Create mask from outline contour

    Parameters
    ----------
    line: array-like (N, 2)
    x, y: 1-D grid coordinates (input for meshgrid)

    Returns
    -------
    mask : 2-D boolean array (True inside)

    Examples
    --------
    >>> from shapely.geometry import Point
    >>> poly = Point(0,0).buffer(1)
    >>> x = np.linspace(-5,5,100)
    >>> y = np.linspace(-5,5,100)
    >>> mask = outline_to_mask(poly.boundary, x, y)
    
    Modified from https://gist.github.com/perrette/a78f99b76aed54b6babf3597e0b331f8
    
    """
    import matplotlib.path as mplp
    mpath = mplp.Path(line)
    X, Y = np.meshgrid(x, y)
    points = np.array((X.flatten(), Y.flatten())).T
    mask = mpath.contains_points(points).reshape(X.shape)
    return mask


def _grid_bbox(x, y):
    dx = dy = 0
    return x[0]-dx/2, x[-1]+dx/2, y[0]-dy/2, y[-1]+dy/2

def _bbox_to_rect(bbox):
    l, r, b, t = bbox
    return Polygon([(l, b), (r, b), (r, t), (l, t)])

def shp_mask(shp, x, y, m=None):
    """
    Use recursive sub-division of space and shapely contains method to create a raster mask on a regular grid.

    Parameters
    ----------
    shp : shapely's Polygon (or whatever with a "contains" method and intersects method)
    x, y : 1-D numpy arrays defining a regular grid
    m : mask to fill, optional (will be created otherwise)

    Returns
    -------
    m : boolean 2-D array, True inside shape.

    Examples
    --------
    >>> from shapely.geometry import Point
    >>> poly = Point(0,0).buffer(1)
    >>> x = np.linspace(-5,5,100)
    >>> y = np.linspace(-5,5,100)
    >>> mask = shp_mask(poly, x, y)
    """
    rect = _bbox_to_rect(_grid_bbox(x, y))
    
    if m is None:
        m = np.zeros((y.size, x.size), dtype=bool)
               
    if not shp.intersects(rect):
        m[:] = False
    
    elif shp.contains(rect):
        m[:] = True
    
    else:
        k, l = m.shape
        
        if k == 1 and l == 1:
            m[:] = shp.contains(Point(x[0], y[0]))
            
        elif k == 1:
            m[:, :l//2] = shp_mask(shp, x[:l//2], y, m[:, :l//2])
            m[:, l//2:] = shp_mask(shp, x[l//2:], y, m[:, l//2:])
            
        elif l == 1:
            m[:k//2] = shp_mask(shp, x, y[:k//2], m[:k//2])
            m[k//2:] = shp_mask(shp, x, y[k//2:], m[k//2:])
        
        else:
            m[:k//2, :l//2] = shp_mask(shp, x[:l//2], y[:k//2], m[:k//2, :l//2])
            m[:k//2, l//2:] = shp_mask(shp, x[l//2:], y[:k//2], m[:k//2, l//2:])
            m[k//2:, :l//2] = shp_mask(shp, x[:l//2], y[k//2:], m[k//2:, :l//2])
            m[k//2:, l//2:] = shp_mask(shp, x[l//2:], y[k//2:], m[k//2:, l//2:])
        
    return m

In [7]:
def island_properties(islands, smooth = True):
    '''
    Identifies islands and calculates island morphological properties
    
    Input:
    -------
    islands: raster of land (1) vs water
    smooth: boolean for median filter to simplify island mask
    
    Output:
    --------
    A list containing:
    island_IMG: array of islands by label
    
    If properties = True, also contains:
    island_props: dictionary of island properties, by label
    ecdf: cumulative probability distributions of island size
    edgedist_IMG: array of islands with calculated distances from edges
    edgedist_hist: histogram of edge distances
    
    '''

    tot_area = islands.sum()

    if smooth: 
        islands_filt = signal.medfilt2d(islands.astype(float), 3)
    else:
        islands_filt = islands.astype(float)

    islandmap, N = ndimage.label(islands_filt)
    islandmap = islandmap.astype('int')
    
    island_props = {}
    
    # for island properties
    rps = skimage.measure.regionprops(islandmap, cache=False)

    Li = [r.major_axis_length for r in rps if r.minor_axis_length > 0]
    Wi = [r.minor_axis_length for r in rps if r.minor_axis_length > 0]
    Pi = [r.perimeter for r in rps if r.minor_axis_length > 0]
    Ai = [r.area for r in rps if r.minor_axis_length > 0]
    label = [r.label for r in rps if r.minor_axis_length > 0]

    island_area = [float(a) / tot_area for a in Ai]
    island_aspect_ratio = [Li[n] / Wi[n] for n in range(len(Li))]
    island_shape_factor = [Pi[n] / np.sqrt(Ai[n]) for n in range(len(Li))]
    
    
    bad_labels = [int(i) for i in np.unique(islandmap) if i not in label]

    for i in bad_labels:
        
        islandmap[islandmap == i] = 0

    
    island_props['major_axis'] = Li
    island_props['minor_axis'] = Wi
    island_props['perimeter'] = Pi
    island_props['area'] = island_area
    island_props['aspt_ratio'] = island_aspect_ratio
    island_props['shp_factor'] = island_shape_factor
    island_props['label'] = label
    
    return islandmap, island_props

In [8]:
preprocess_flag = False

raster_filepath = '_input/gangeschan.tif'
network_filepath = '_input/network.shp'

In [10]:
raster, (r_xmin, r_xmax, r_ymin, r_ymax, r_xres, r_yres) = read_tiff_as_array(raster_filepath, get_info= True)
islandmap = (raster > 1) * 1.

In [11]:
# get islands from network shapefile

c = fiona.open(network_filepath)

network_lines = MultiLineString([shape(pol['geometry']) for pol in c])
network_widths = [pol['properties']['Width'] for pol in c]
network_azimuths = [azimuth(Point(l.coords[0]),Point(l.coords[-1])) for l in network_lines]

result, dangles, cuts, invalids = polygonize_full(network_lines)
islands = MultiPolygon([i for i in result if i.area > r_xres * r_xres * 10])

c = None


# find islands with interior polygons
# first number is index of island with interior polygons
# second number is index of interior island

get_contained_islands = preprocess_flag

if get_contained_islands:
    
    contained_islands = []

    for i in range(len(islands)):

        # if island has an inner ring
        if not islands[i].exterior.equals(islands[i].boundary):

            # check all other islands
            for j in range(len(islands)):
                if i != j:

                    # check if one is inside the other
                    inside = islands[j].within(Polygon(islands[i].exterior))

                    if inside:
                        contained_islands.append([i,j])

    pickle.dump(contained_islands, open( '_output/contained_islands' + '.p', "wb" ) )
    
else:
    
    contained_islands = pickle.load( open( '_output/contained_islands.p', "rb" ) )


bad_islands = set([i[1] for i in contained_islands])
holey_islands = set([i[0] for i in contained_islands])

new_islands = [Polygon(islands[i].exterior) if i in holey_islands else islands[i] for i in range(len(islands))]
new_islands = [new_islands[i] for i in range(len(new_islands)) if i not in bad_islands]

islands = MultiPolygon(new_islands)



In [12]:
get_bounding_channels = preprocess_flag

if get_bounding_channels:

    # midpoints of network lines, with buffers
    midpts = [l.interpolate(0.5, normalized=True).buffer(5) for l in network_lines]

    bounds = []
    interior_channels = []

    # check if line midpoints intersect the island outlines
    #(to identify which lines make up each island)
    for polygon in islands:
    
        touch = [i for i,l in enumerate(midpts) if polygon.exterior.intersects(l)]
        bounds.append(touch)
        
        touch = [i for i,l in enumerate(midpts) if polygon.contains(l)]
        interior_channels.append(touch)

    pickle.dump(bounds, open( '_output/island_boundary_channels' + '.p', "wb" ) )
    pickle.dump(interior_channels, open( '_output/island_interior_channels' + '.p', "wb" ) )

else:    
    bounds = pickle.load( open( '_output/island_boundary_channels.p', "rb" ) )
    interior_channels = pickle.load( open( '_output/island_interior_channels.p', "rb" ) )
    
flat_bounds = np.unique([item for sublist in bounds for item in sublist])

In [13]:
# create shapefile and raster of the network islands

create_island_GIS = preprocess_flag

if create_island_GIS:

    channel_bounds = [network_lines[i] for i in flat_bounds]

    create_shapefile_from_shapely_multi(MultiLineString(channel_bounds),
                                        '_output/islands_polygons.shp',
                                        buffer_width = r_xres/2.)


    create_tiff_from_shapefile('_output/islands_polygons.shp',
                               '_output/islands_polygons.tif',
                               '_input/gangeschan.tif')

In [14]:
# Etch the network islands onto the islandmap found from the image
# to highlight channels that are not seen as continuous in the image
#
# Then run island_properties again and get island properties

get_new_island_properties = preprocess_flag
pickle_island_properties = preprocess_flag

if get_new_island_properties:

    islandmap1, island_props0 = island_properties(islandmap, smooth = False)

    # remove invalid labels (set with min_valid_label) and
    # then enough of the smallest island labels to have
    # the same number as islands from the network

    min_valid_label = 3

    num_id_islands = len(island_props0['label'])
    expected_num_islands = len(islands)

    diff_number = num_id_islands - expected_num_islands - min_valid_label


    # label must be less than min_valid_label
    # then remove the smallest diff_number islands

    area_sort = np.argsort(island_props0['area'])

    flag = False
    dn = 0

    while (not flag) & (dn <= min_valid_label):

        dn += 1
        small_island_id = [i for i in area_sort[:diff_number+dn] if island_props0['label'][i] >= min_valid_label]
        
        flag = len(small_island_id) == diff_number + dn
        
        


    keep_props_id = [i for i in area_sort[diff_number+dn:] if island_props0['label'][i] >= min_valid_label]

    island_props = {}

    for f,v in island_props0.items():
        island_props[f] = np.array(island_props0[f])[keep_props_id]
        
    bad_labels = [int(i) for i in np.unique(islandmap1) if i not in island_props['label']]

    for i in bad_labels:
        islandmap1[islandmap1 == i] = 0
        
        
    if pickle_island_properties:
    
        pickle.dump(island_props, open( '_output/island_props' + '.p', "wb" ) )
        SaveRaster(islandmap1, '_output/island_map_HD.tif', r_xmin, r_ymin, r_xres)
        
else:
    
    island_props = pickle.load( open( '_output/island_props.p', "rb" ) )
    islandmap1 = read_tiff_as_array('_output/island_map_HD.tif')

In [15]:
get_channel_props = preprocess_flag

if get_channel_props:

    # associate channel properties to islands

    network_min_widths = np.zeros((len(islands),))
    network_avg_widths = np.zeros((len(islands),))
    network_max_widths = np.zeros((len(islands),))

    for n in range(len(islands)):

        i = islands[n]

        # channels added in open water to close island polygons have width 9999
        channels = [network_lines[b] for b in bounds[n] if network_widths[b] != 9999]
        widths = [network_widths[b] for b in bounds[n] if network_widths[b] != 9999]
        lengths = [c.length for c in channels]


        tot_length = sum(lengths)
        avg_width = sum([widths[b] * lengths[b] for b in range(len(widths))]) / tot_length
        max_width = max(widths)
        min_width = min(widths)


        network_min_widths[n] = min_width
        network_avg_widths[n] = avg_width
        network_max_widths[n] = max_width
        
    pickle.dump([network_min_widths, network_avg_widths, network_max_widths],
                open( '_output/channel_widths' + '.p', "wb" ) )
    
else:
    
    network_min_widths, network_avg_widths, network_max_widths = pickle.load(
                            open( '_output/channel_widths.p', "rb" ) )

In [16]:
get_poly_label = preprocess_flag

# poly_label is an array of the size of islands with the label from islandmap1
#(and therefore the connection to island_props)

if get_poly_label:

    poly_label, poly_index, bad_pts = get_value_of_raster_within_polygons(islandmap1.astype('int'),
                                                                          islands,
                                                                          r_xmin, r_xmax,
                                                                          r_ymin, r_ymax,
                                                                          min_valid_label=3)
    poly_label = np.array(poly_label)

    pickle.dump([poly_label, poly_index], open( '_output/poly_label' + '.p', "wb" ) )
    
else:
    
    poly_label, poly_index = pickle.load( open( '_output/poly_label.p', "rb" ) )

In [17]:
get_poly_metrics = preprocess_flag

if get_poly_metrics:

    num_ox = []

    for i in islands:

        oxs = 0
        for j in i.interiors:

            try:
                a = Polygon(j).buffer(-20).area

                if a > 40000:
                    oxs += 1
            except:
                pass

        num_ox.append(oxs)



    interior_lengths = [sum([network_lines[j].length for j in interior_channels[i]]) if len(interior_channels[i])>0 else 0 for i in range(len(shp_islands))] 

    perimeter = np.array([i.boundary.length for i in shp_islands])
    wetted_perimeter = perimeter + 2 * np.array(interior_lengths)   
    area = np.array([i.area for i in shp_islands])
    perimeter_convex_hull = np.array([i.convex_hull.exterior.length for i in shp_islands])
    area_convex_hull = np.array([i.convex_hull.area for i in shp_islands])

    a = np.array(map(Polygon_axes, shp_islands))
    minor_axis = a[:,0]
    major_axis = a[:,1]
    poly_orientation = a[:,2]
    aspect_ratio = major_axis / minor_axis

    circularity = 4 * np.pi * area / perimeter**2
    equivalent_area_diameter = np.sqrt((4 / np.pi) * area)
    perimeter_equivalent_diameter = area / np.pi
    solidity = area / area_convex_hull
    concavity = area_convex_hull - area
    convexity = perimeter_convex_hull / perimeter
    dry_shape_factor = perimeter / np.sqrt(area)
    wet_shape_factor = wetted_perimeter / np.sqrt(area)

    poly_metrics = {'p_area': area,
                    'p_perim': perimeter,
                    'p_wetperim': wetted_perimeter,
                    'p_ch_area': area_convex_hull,
                    'p_ch_perim': perimeter_convex_hull,
#                     'p_max_rad': maximum_inscribed_circle_radius,
                    'p_major_ax': major_axis,
                    'p_minor_ax': minor_axis,
                    'p_asp_rat': aspect_ratio,
                    'p_orient': poly_orientation,
                    'p_circ': circularity,
                    'p_eq_a_dia': equivalent_area_diameter,
                    'p_p_eq_dia': perimeter_equivalent_diameter,
                    'p_solidity': solidity,
                    'p_concav': concavity,
                    'p_convex': convexity,
                    'p_d_shapef': dry_shape_factor,
                    'p_w_shapef': wet_shape_factor,
                    'p_label': shp_islands_label,
                    'p_num_ox': num_ox}

    pickle.dump(poly_metrics, open( '_output/island_shp_metrics' + '.p', "wb" ) )

else:

    poly_metrics = pickle.load( open( '_output/island_shp_metrics.p', "rb" ) )

In [18]:
shp_islands = islands

In [38]:
calc_sinuosity = preprocess_flag


if calc_sinuosity:

    sinuosity_all = np.ones((len(shp_islands), 3))
    
    for cn, c_dist in enumerate([500, 1000, 1500]):

    
        for ni,i in enumerate(shp_islands):
            sinuosity = np.ones((len(bounds[ni]),))

        
            for n,b in enumerate(bounds[ni]):
                line = network_lines[b]

                
                if line.length > c_dist:

                    l_dist = []

                    for s in range(0, c_dist, 100):

                        coords = []

                        for d in np.arange(s, line.length+1, c_dist):  
                            coords.append(line.interpolate(d).coords[0])

                        diff = np.diff(np.array(coords), axis=0)
                        l_dist += list(np.sqrt(diff[:,0]**2 + diff[:,1]**2))


                        
                    rline = LineString(list(line.coords)[::-1])    
                    for s in range(0, c_dist, 100):

                        coords = []

                        for d in np.arange(s, rline.length+1, c_dist):  
                            coords.append(rline.interpolate(d).coords[0])

                        diff = np.diff(np.array(coords), axis=0)
                        l_dist += list(np.sqrt(diff[:,0]**2 + diff[:,1]**2))

                        
                        
                    sin = np.median(c_dist / np.array(l_dist))
                    sinuosity[n] = sin


            sinuosity_all[ni,cn] = np.mean(sinuosity)


    pickle.dump(sinuosity_all, open( '_output/bound_channel_sinuosity_vals' + '.p', "wb" ))

    
else:
    
    sinuosity_all = pickle.load( open( '_output/bound_channel_sinuosity_vals.p', "rb" ) )

    

In [39]:
# find angle that outflow channel makes with boundary

find_outflow_angles = preprocess_flag

if find_outflow_angles:

    num_outflow = np.zeros((len(islands),))

    for n in range(len(islands)):

        lines = [i for i in interior_channels[n]]

        # interior channels that touch the boundary
        outflow = []

        for l in lines:
            if islands[n].exterior.touches(network_lines[l]):
                outflow.append(l)

        angle = []
        num_out = 0

        for l in outflow:

            inside_line = shp_islands[n].intersection(network_lines[l])
            node = shp_islands[n].boundary.intersection(network_lines[l])

            if node.type is 'GeometryCollection':
                node = shp_islands[n].boundary.intersection(network_lines[l])

            if inside_line.type is 'LineString':
                num_out += 1

        
        num_outflow[n] = num_out

    pickle.dump(num_outflow, open( '_output/outflow_number' + '.p', "wb" ) )
    
else:
    num_outflow = pickle.load( open( '_output/outflow_number.p', "rb" ) )

In [41]:
get_bound_angles = preprocess_flag

if get_bound_angles:

    all_channel_angles = []

    for isl in range(len(islands)):

        lines = [network_lines[l] for l in bounds[isl]]
        num_lines = [l for l in bounds[isl]]

        channel_angles = []


        for l1,l2 in itertools.combinations(range(len(lines)),2):

            if lines[l1].touches(lines[l2]):

                node_ = lines[l1].intersection(lines[l2])

                if node_.type is 'Point':
                    node_ = [node_]

                for node in node_:

                    third_line = [n for n,i in enumerate(network_lines) if (i.intersects(node.buffer(20)) and
                                                                            n not in [num_lines[l1],num_lines[l2]])]

                    if len(third_line) > 0:

                        intersecting_lines = third_line + [num_lines[l1], num_lines[l2]]


                        line0 = network_lines[intersecting_lines[0]]
                        line1 = network_lines[intersecting_lines[1]]
                        line2 = network_lines[intersecting_lines[2]]


                        try:

                            buffer_l = 200

                            intersect0 = line0.intersection(node.buffer(buffer_l).exterior)
                            if intersect0.type is 'MultiPoint':
                                intersect0 = intersect0[0]
                            az0 = azimuth(node, intersect0)

                            intersect1 = line1.intersection(node.buffer(buffer_l).exterior)
                            if intersect1.type is 'MultiPoint':
                                intersect1 = intersect1[0]
                            az1 = azimuth(node, intersect1)

                            intersect2 = line2.intersection(node.buffer(buffer_l).exterior)
                            if intersect2.type is 'MultiPoint':
                                intersect2 = intersect2[0]
                            az2 = azimuth(node, intersect2)

                        except:

                            buffer_l = np.min([line0.simplify(10000).length * 0.8,
                            line1.simplify(1000).length * 0.8,
                            line2.simplify(1000).length * 0.8,
                            200])

                            intersect0 = line0.intersection(node.buffer(buffer_l).exterior)
                            if intersect0.type is 'MultiPoint':
                                intersect0 = intersect0[0]
                            az0 = azimuth(node, intersect0)

                            intersect1 = line1.intersection(node.buffer(buffer_l).exterior)
                            if intersect1.type is 'MultiPoint':
                                intersect1 = intersect1[0]
                            az1 = azimuth(node, intersect1)

                            intersect2 = line2.intersection(node.buffer(buffer_l).exterior)
                            if intersect2.type is 'MultiPoint':
                                intersect2 = intersect2[0]
                            az2 = azimuth(node, intersect2)

                        channel_angles.append([az0, az1, az2])

        all_channel_angles.append(channel_angles)

    angle_stats = np.zeros((len(all_channel_angles), 5))

    for n, a in enumerate(all_channel_angles):

        if len(a) > 0:

            angle_differences_ = np.zeros((len(a), 3))

            for nn, r in enumerate(a):

                az0, az1, az2 = r

                az01 = np.abs(az0 - az1)
                az01 = az01 if az01 < 180 else az01 - 180
                az02 = np.abs(az0 - az2)
                az02 = az02 if az02 < 180 else az02 - 180
                az12 = 360 - az01 - az02


                angle_differences_[nn,:] = [az01,az02,az12]


            diff = angle_differences_.flatten()
            stats = [np.min(diff), np.max(diff), np.mean(diff), np.median(diff), np.std(diff)]

            angle_stats[n,:] = stats
        
    pickle.dump(angle_stats, open( '_output/bound_channel_angle_stats' + '.p', "wb" ) )
    pickle.dump(all_channel_angles, open( '_output/bound_channel_all_angles' + '.p', "wb" ) )
    
else:
    
    angle_stats = pickle.load( open( '_output/bound_channel_angle_stats.p', "rb" ) )
    all_channel_angles = pickle.load( open( '_output/bound_channel_all_angles.p', "rb" ) )

angle_differences = []
angle_stats = np.zeros((len(all_channel_angles), 5))

for n, a in enumerate(all_channel_angles):

    if len(a) > 0:

        angle_differences_ = np.zeros((len(a), 3))

        for nn, r in enumerate(a):

            az0, az1, az2 = r

            az01 = np.abs(az0 - az1)
            az01 = az01 if az01 < 180 else az01 - 180
            az02 = np.abs(az0 - az2)
            az02 = az02 if az02 < 180 else az02 - 180
            az12 = 360 - az01 - az02


            angle_differences_[nn,:] = [az01,az02,az12]

        angle_differences.append(angle_differences_)

        diff = angle_differences_.flatten()
        stats = [np.min(diff), np.max(diff), np.mean(diff), np.median(diff), np.std(diff)]

        angle_stats[n,:] = stats

In [42]:
angle_stats2 = np.zeros((len(all_channel_angles), 3))

for n, a in enumerate(all_channel_angles):

    if len(a) > 0:

        sort = [i.sort() for i in a]

        small_angle = [i[0] for i in a]
        mid_angle = [i[1] for i in a]
        large_angle = [i[2] for i in a]

        angle_stats2[n,:] = [np.mean(small_angle), np.mean(mid_angle), np.mean(large_angle)]

In [43]:
calc_edge_dist = preprocess_flag

if calc_edge_dist:

    edgedists = np.zeros((len(shp_islands),))

    for n,s in enumerate(shp_islands):

        print n

        cellsize = 10

        minx, miny, maxx, maxy = s.envelope.bounds

        if ((maxx - minx) / cellsize > 1000) or ((maxy - miny) / cellsize > 1000):
            cellsize = 100



        minx = np.floor(minx) - 1 * cellsize
        maxx = np.ceil(maxx) + 2 * cellsize
        miny = np.floor(miny) - 1 * cellsize
        maxy = np.ceil(maxy) + 2 * cellsize

        x = np.arange(minx, maxx , cellsize)
        y = np.arange(miny, maxy , cellsize)


        mask = outline_to_mask(s.exterior, x, y)
        distmap = morphology.distance_transform_edt(mask)

        edgedists[n] = distmap.max() * cellsize


    pickle.dump(edgedists, open( '_output/edge_distances' + '.p', "wb" ) )

else:
    
    edgedists = pickle.load( open( '_output/edge_distances.p', "rb" ))

In [49]:
   

# save island_props to a shapefile

new_props = {'min_width':network_min_widths,
    'avg_width':network_avg_widths,
             'edge_d2': edgedists,
             'max_width':network_max_widths,
#              'o_ang_min': outflow_stats[:,0],
#              'o_ang_max': outflow_stats[:,1],
#              'o_ang_mean': outflow_stats[:,2],
#              'o_ang_med': outflow_stats[:,3],
#              'o_ang_std': outflow_stats[:,4],
             'out_numbr': num_outflow,
#              'ch_ang_min': angle_stats[:,0],
#              'ch_ang_max': angle_stats[:,1],
#              'ch_ang_med': angle_stats[:,3],
#              'ch_ang_std': angle_stats[:,4],
             'sin500': sinuosity_all[:,0],
            'sin1000': sinuosity_all[:,1],
            'sin1500': sinuosity_all[:,2],
#              'ang0': angle_stats2[:,0],
#             'ang1': angle_stats2[:,1],
#             'ang2': angle_stats2[:,2],
            }

all_keys = new_props.keys() + poly_metrics.keys()



island_prop_array = np.zeros((len(islands),len(all_keys)+1))

for i in island_props['label']:
    
    loc = np.where(np.array(island_props['label']) == i)[0]
    loc_poly = np.where(poly_label == i)[0]
    
    if (len(loc) > 0) & (len(loc_poly) > 0):
        
        loc = loc[0]
        props1 = []
        props2 = [new_props[f][loc_poly[0]] for f in new_props.keys()]
        props3 = [poly_metrics[f][loc_poly[0]] for f in poly_metrics.keys()]
        
        
        for p in loc_poly:
            
            island_prop_array[p,:-1] = props1 + props2 + props3

island_prop_array[np.isnan(island_prop_array)] = 0

island_prop_array[:,-1] = range(0,len(islands))
all_keys.append('id')


pickle.dump(island_prop_array, open( '_output/island_prop_array' + '.p', "wb" ) )




field_type = {}

for k in all_keys:
    field_type[k] = ogr.OFTReal
    
    
bad_islands = np.where(island_prop_array[:,0] == 0)[0]

island_prop_array = island_prop_array[island_prop_array[:,0] > 0,:] # keep only the lines that have a label
shp_islands2 = [islands[i] for i in range(len(islands)) if i not in bad_islands]
    

fields = {}    
for n,f in enumerate(all_keys):
    fields[f] = island_prop_array[:,n]
    

create_shapefile_from_shapely_multi(shp_islands2,
                                    '_output/islands_properties.shp',
                                    fields = fields,
                                    field_type = field_type)
